In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import pickle
import torch
# import cuml

from scipy.stats import pearsonr
from sklearn.feature_selection import chi2
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix as cf_matrix

In [2]:
if torch.cuda.is_available():
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8500
__Number CUDA Devices: 3
__CUDA Device Name: Tesla V100-PCIE-32GB
__CUDA Device Total Memory [GB]: 34.089926656


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_csv(r"/maps/hdir/gsw508/Thesis/Data/ParlSpeech_V2/Corp_Folketing_V2.csv")

In [5]:
df

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK
...,...,...,...,...,...,...,...,...,...,...,...
772175,2018-12-20,2. (sidste) behandling af beslutningsforslag ...,330,Pia Kjærsgaard,NaN,NaN,True,28,Der stemmes om forslagets endelige vedtagelse....,DK-Folketing,DNK
772176,2018-12-20,2. (sidste) behandling af beslutningsforslag ...,331,Pia Kjærsgaard,NaN,NaN,True,22,Der er ikke stillet ændringsforslag. Er der n...,DK-Folketing,DNK
772177,2018-12-20,2. (sidste) behandling af beslutningsforslag ...,332,Pia Kjærsgaard,NaN,NaN,True,37,Der stemmes om forslagets endelige vedtagelse....,DK-Folketing,DNK
772178,2018-12-20,2. (sidste) behandling af beslutningsforslag ...,333,Pia Kjærsgaard,NaN,NaN,True,61,Der er ikke mere at foretage i dette møde. Fo...,DK-Folketing,DNK


In [6]:
cmp_df = pd.read_csv("/maps/hdir/gsw508/Thesis/Data/MPDataset_MPDS2022a.csv")

In [7]:
cmp_df['countryname'].unique()

array(['Sweden', 'Norway', 'Denmark', 'Finland', 'Iceland', 'Belgium',
       'Netherlands', 'Luxembourg', 'France', 'Italy', 'Spain', 'Greece',
       'Portugal', 'Germany', 'Austria', 'Switzerland', 'United Kingdom',
       'Northern Ireland', 'Ireland', 'Malta', 'Cyprus', 'United States',
       'Canada', 'Australia', 'New Zealand', 'Japan', 'Israel',
       'Sri Lanka', 'Turkey', 'Albania', 'Armenia', 'Azerbaijan',
       'Belarus', 'Bosnia-Herzegovina', 'Bulgaria', 'Croatia',
       'Czech Republic', 'Estonia', 'Georgia',
       'German Democratic Republic', 'Hungary', 'Latvia', 'Lithuania',
       'North Macedonia', 'Moldova', 'Montenegro', 'Poland', 'Romania',
       'Russia', 'Serbia', 'Slovakia', 'Slovenia', 'Ukraine',
       'South Korea', 'Mexico', 'South Africa'], dtype=object)

In [8]:
cmp_df.loc[(cmp_df['countryname'] == 'Denmark'), 'partyabbrev'].value_counts()

RV     28
V      28
KF     28
SD     28
SF     22
DKP    17
RF     17
KrF    13
CD     12
FP     11
EL      8
VS      8
DF      7
DU      6
LC      2
FK      1
DS      1
K       1
NY      1
Name: partyabbrev, dtype: int64

In [9]:
cmp_df.loc[(cmp_df['countryname'] == 'Denmark'), 'partyname'].value_counts()

Danish Social-Liberal Party    28
Liberals                       28
Conservative People’s Party    28
Social Democratic Party        28
Socialist People’s Party       22
Danish Communist Party         17
Justice Party                  17
Christian People’s Party       13
Centre Democrats               12
Progress Party                 11
Left Socialist Party            8
Red-Green Unity List            8
Danish People’s Party           7
Independents’ Party             6
Liberal Alliance                3
Liberal Centre                  2
Alternativ                      2
Danish Union                    1
Common Course                   1
Christian Democrats             1
New Alliance                    1
The New Right                   1
Name: partyname, dtype: int64

In [10]:
party_df = df.loc[df['party'].isna() == False]

In [11]:
cmp_df.loc[(cmp_df['countryname'] == 'Denmark') & (cmp_df['partyname'] == 'Centre Democrats'), 'parfam']

354    60
365    60
376    60
387    60
398    60
409    60
419    60
427    60
435    60
444    60
453    60
471    60
Name: parfam, dtype: int64

In [12]:
party_df['party'].value_counts()

S      92651
V      92381
DF     58199
EL     54971
SF     47481
KF     41518
RV     34322
LA     14804
ALT     5819
KRF     4769
CD      3363
FP      1693
KD      1460
UP      1198
IA       933
FRI      918
UFG      891
T        634
NY       591
SIU      406
SP       145
NQ        85
FF        44
LH         9
JF         4
Name: party, dtype: int64

In [13]:
# Partyfam 10, 20, 30
left_list = ['SD', 'SF', 'EL', 'ALT']

# Parties from Greenland and Faroe Islands are not categorized by the CMP. They will be considered partyfam 90 - ethnic.

In [14]:
party_df.loc[party_df['party'] == 'SIU']

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
76285,2001-12-06,1) Forhandling om redegørelse nr. R 1:,177,Lars Emil Johansen,SIU,6651.0,False,1137,Tak. Efter at have været uden for Folketinget ...,DK-Folketing,DNK
76302,2001-12-06,1) Forhandling om redegørelse nr. R 1:,194,Lars Emil Johansen,SIU,6651.0,False,137,"Statsministeren nævnte, at 1951-aftalen skam v...",DK-Folketing,DNK
80792,2002-02-01,1) Første behandling af lovforslag nr. L 70: F...,310,Lars-Emil Johansen,SIU,6651.0,False,2345,For en enhver ny regering er den første finans...,DK-Folketing,DNK
80816,2002-02-01,1) Første behandling af lovforslag nr. L 70: F...,334,Lars-Emil Johansen,SIU,6651.0,False,2343,For en enhver ny regering er den første finans...,DK-Folketing,DNK
89139,2002-04-05,5) Forespørgsel nr. F 13: Forespørgsel til ude...,55,Lars-Emil Johansen,SIU,6651.0,False,475,Endnu en gang indgår Grønlands betydning for D...,DK-Folketing,DNK
...,...,...,...,...,...,...,...,...,...,...,...
659630,2016-06-01,Forespørgsel nr. F 40: Forespørgsel til stat...,1092,Aleqa Hammond,SIU,6651.0,False,119,"Jeg tror helt sikkert, at det er af stor inter...",DK-Folketing,DNK
659658,2016-06-01,Forespørgsel nr. F 40: Forespørgsel til stat...,1120,Aleqa Hammond,SIU,6651.0,False,180,"Når vi hører oplæggene, både fra Grønland og F...",DK-Folketing,DNK
659662,2016-06-01,Forespørgsel nr. F 40: Forespørgsel til stat...,1124,Aleqa Hammond,SIU,6651.0,False,71,Godt nok er Færøerne med i det danske kongerig...,DK-Folketing,DNK
659768,2016-06-01,Forespørgsel nr. F 40: Forespørgsel til stat...,1230,Aleqa Hammond,SIU,6651.0,False,97,Rigsrevisionen fremkom med en meget skarp krit...,DK-Folketing,DNK


In [15]:
party_df['left'] = 0

/tmp/ipykernel_21902/3910170133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  party_df['left'] = 0


In [16]:
party_df.loc[party_df['party'].isin(left_list), 'left'] = 1

In [17]:
party_df['left'].value_counts()

0    351018
1    108271
Name: left, dtype: int64

In [18]:
party_df = party_df.loc[party_df['terms'] > 40]

In [19]:
party_df.to_json('den_party_df.json')

socio-economic (class); religious (church–state); ethnic (center–periphery); urban–rural; post-materialist; and foreign policy

In [19]:
def preprocess(text):
    'Creating preprocessing pipeline'

    # Lowercasing
    text = text.lower()

    # Removing punctuation
    text = re.sub(r'[()\.\,\?\!\"\'\-\`\:\;\\\%\*\[\]]+','', text)

    return text

In [20]:
nlp = spacy.load("xx_sent_ud_sm")
def token(text):
    doc = nlp(text)
    return [token.text for token in doc]

In [21]:
df_preprocessed = party_df['text'].apply(lambda x: preprocess(x))

In [22]:
df_preprocessed

2         for 25 år siden sagde et flertal i befolkninge...
4         vi står over for en meget afgørende folketings...
6         hr torben lund taler om indgreb imod det stige...
7         ligesom i åbningstalen kom udlændingeafsnittet...
8         jeg kan forstå at udlændingepolitikken er kraf...
                                ...                        
772147    vi bakker op om det her forslag fordi vi synes...
772149    al respekt for fru lotte rods opfordring men m...
772151    man kan jo grundlæggende have to tilgange til ...
772165    om lidt vil der være et flertal som stemmer de...
772169    siden 2016 er 62 børn blev udvist selv om en a...
Name: text, Length: 438115, dtype: object

In [23]:
# Vectorizing 
vectorizer = TfidfVectorizer(min_df= 0.0001) # Default N-gram range is (1,1)
den_features = vectorizer.fit_transform(df_preprocessed)

In [24]:
pickle.dump(vectorizer, open("den_vectorizer.pickle", "wb"))
pickle.dump(den_features, open("den_features.pickle", "wb"))